In [1]:
import pandas as pd
import numpy as np

In [2]:
# load csv containing zipcodes and clean the file before merging
zipcodes = pd.read_csv('zipcodes.csv')
zipcodes['City'] = zipcodes.apply(lambda row: row['City'] if row['County'] != 'Queens' else row['County'], axis = 1) 
zipcodes = zipcodes.ix[:, :2]
zipcode_key = zipcodes.set_index('Zip Code').T.to_dict('list')

In [3]:
df_rent = pd.read_csv('streeteasy_rent_clean_geo.csv')
df_rent = df_rent[df_rent['fill_sqft'] != 1]
df_rent = df_rent[df_rent['fill_baths'] != 1]

drops = []
for idx, column in enumerate(df_rent.columns.values):
    #print idx, column
    if 'Unnamed' in column or 'fill' in column:
        drops.append(idx)
df_rent.drop(df_rent.columns[drops], 1, inplace = True)

In [4]:
def get_borough(zipcode):
    try:
        return zipcode_key[int(zipcode)][0]
    except:
        return np.nan

df_rent['Borough'] = df_rent['Zipcode'].apply(lambda zipcode: get_borough(zipcode))
df_rent['Sqft_sq'] = df_rent['Sqft'].apply(lambda sqft: sqft ** 2.0)

In [5]:
df_rent.Borough = df_rent.Borough.replace(to_replace = 'New York', value = 'Manhattan')

In [6]:
df_rent['Borough'].value_counts()

Manhattan        3410
Brooklyn         1398
Queens            482
Bronx              58
Staten Island      45
Name: Borough, dtype: int64

In [7]:
df_rent.columns.values

array(['Address', 'Architect', 'Baths', 'Beds', 'Bike Room',
       'Building Amenities', 'Building Class', 'Built Year',
       'Cats Only - No Dogs', 'Cats and Dogs Allowed',
       "Children's Playroom", 'City', 'City Council', 'Cold Storage',
       'Community', 'Community Recreation Facilities', 'Concierge',
       'Crime', 'Deck', 'Developer', 'Doorman', 'Elevator',
       'Full-time Doorman', 'Garage Parking', 'Garden', 'Green Building',
       'Gym', 'Highlights', 'Hot Tub', 'Interiors', 'Land Lease',
       'Laundry in Building', 'Live-in Super', 'Media Room',
       'NYC Storm Zone 1', 'NYC Storm Zone 2', 'NYC Storm Zone 3',
       'NYC Storm Zone 4', 'NYC Storm Zone 5', 'NYC Storm Zone 6',
       'Package Room', 'Parking Available', 'Part-time Doorman', 'Patio',
       'Pets Allowed', 'Police', 'Price', 'Roof Deck', 'School District',
       'SchoolScore', 'Smoke-free', 'Sqft', 'State', 'Storage Available',
       'Stories', 'Street', 'Swimming Pool', 'Transportation', 'Unit

In [8]:
df_rent_clean = df_rent.drop(['Architect', 'Building Amenities', 'Community', 'City Council', 
            'Developer', 'Highlights', 'Transportation', 'Street', 'State'], axis=1)
df_rent_clean = df_rent_clean[df_rent_clean['Sqft'] < 60000]

In [9]:
df_rent_clean.to_csv('rent_clean_dataset.csv')

In [12]:
# store the predictor dataset for visualization in tableau
df_rent_clean.to_excel('rent_visualization_data/rent_full_dataset.xlsx')